In [43]:
def discretize_bearing(bearing):
  """
  방위각(0~360도)을 0, 90, 180, 270도 중 가장 가까운 값으로 정제합니다.
  """
  # 북쪽 기준을 위해 22.5도를 더하고, 360도로 나눈 나머지를 취합니다.
  # 이렇게 하면 북쪽(0도)을 기준으로 22.5~67.5도는 45도 범위에 들어갑니다.
  adjusted_bearing = (bearing + 45) % 360 

  # adjusted_bearing을 90으로 나누어 몇 번째 사분면에 가까운지 확인
  # 예: 10도 -> (10+45)%360 = 55 -> 55/90 = 0.61 -> round(0.61) * 90 = 0
  # 예: 100도 -> (100+45)%360 = 145 -> 145/90 = 1.61 -> round(1.61) * 90 = 180 (X) -> 이 방법은 복잡하고 오류 가능성이 높습니다.

  # 직관적인 방법을 사용합니다.
  
  # 45도로 나누어 몫을 구합니다.
  # 0~45도: 0/90 (북)
  # 45~135도: 90/90 (동)
  # 135~225도: 180/90 (남)
  # 225~315도: 270/90 (서)
  # 315~360도: 360/90 (북)
  
  # 북쪽을 $360^\circ$로 생각하고 조정하여 반올림합니다.
  if bearing > 315 or bearing <= 45:
    return 0 # 북
  elif bearing > 45 and bearing <= 135:
    return 90 # 동
  elif bearing > 135 and bearing <= 225:
    return 180 # 남
  elif bearing > 225 and bearing <= 315:
    return 270 # 서
  else:
    # 이 부분은 발생하지 않아야 하지만, 안전을 위해 0을 반환합니다.
    return 0 


In [44]:


# 예시 테스트:
print(f"방위각 20도 -> 정제 방향: {discretize_bearing(20)}도 (북)")
print(f"방위각 110도 -> 정제 방향: {discretize_bearing(110)}도 (동)")
print(f"방위각 200도 -> 정제 방향: {discretize_bearing(200)}도 (남)")
print(f"방위각 300도 -> 정제 방향: {discretize_bearing(300)}도 (서)")
print(f"방위각 350도 -> 정제 방향: {discretize_bearing(350)}도 (북)")

방위각 20도 -> 정제 방향: 0도 (북)
방위각 110도 -> 정제 방향: 90도 (동)
방위각 200도 -> 정제 방향: 180도 (남)
방위각 300도 -> 정제 방향: 270도 (서)
방위각 350도 -> 정제 방향: 0도 (북)


In [53]:
import math

def calculate_bearing(lat1, lon1, lat2, lon2):
  # 1. 모든 도(Degree) 값을 라디안(Radian)으로 변환
  lat1_rad = math.radians(float(lat1))
  lon1_rad = math.radians(float(lon1))
  lat2_rad = math.radians(float(lat2))
  lon2_rad = math.radians(float(lon2))

  # 경도 차이
  dLon = lon2_rad - lon1_rad

  # 2. 방위각 계산 공식 적용 (arctan2 사용)
  y = math.sin(dLon) * math.cos(lat2_rad)
  x = math.cos(lat1_rad) * math.sin(lat2_rad) - math.sin(lat1_rad) * math.cos(lat2_rad) * math.cos(dLon)
  
  # atan2는 라디안으로 결과를 반환하며, x, y 값을 통해 사분면을 고려함
  bearing_rad = math.atan2(y, x)

  # 3. 라디안을 도(Degree)로 변환
  bearing_deg = math.degrees(bearing_rad)

  # 4. 0 ~ 360도 범위로 조정
  # 결과가 음수일 경우 360을 더해서 양수로 만듦 (예: -90 -> 270)
  final_bearing = (bearing_deg + 360) % 360

  return final_bearing


In [46]:

# 예시: 서울(출발지)에서 부산(도착지) 방향
seoul_lat, seoul_lon = 37.5665, 126.9780
busan_lat, busan_lon = 35.1796, 129.0756

direction = calculate_bearing(seoul_lat, seoul_lon, busan_lat, busan_lon)

print(f"서울에서 부산으로의 이동 방향 (방위각): {direction:.2f}도, 정제{discretize_bearing(direction)}") 
# 결과는 약 154.55도로, 남동쪽 (135도) 방향에 가까움을 알 수 있습니다.

서울에서 부산으로의 이동 방향 (방위각): 144.09도, 정제180


In [ ]:
def calcAngle(originX, originY, destX, destY):
    return discretize_bearing(calculate_bearing(originY, originX, destY, destX))

In [49]:
print("calcAngle=", calcAngle(seoul_lon, seoul_lat, busan_lon, busan_lat))

calcAngle= 180


In [ ]:
import requests

# 2개스페이스로 된 탭으로 보여줍니다.
# URL 및 헤더 정보
url = "https://apis-navi.kakaomobility.com/v1/directions"
headers = {
  "Authorization": "KakaoAK KEY"
}

originX = "127.1027997"
originY = "37.61354475"
destX = "127.099763"
destY = "37.614242"

# 쿼리 파라미터 (Query Parameters)
# curl 명령어의 '?' 뒤에 오는 부분입니다.
params = {
  "origin": f"{originX},{originY},angle={calcAngle(originX, originY, destX, destY)}",
  "destination": f"{destX},{destY}",
  "priority": "DISTANCE", # DISTANCE:최단거리, TIME:최단시간, RECOMMEND:추천
  "car_type": "3", # 1:소형,2:중형,3:대형(33인승이상)
  "summary":"true" #요약정보만
}

try:
  # GET 요청 보내기
  response = requests.get(url, headers=headers, params=params)

  # 응답 확인
  if response.status_code == 200:
    # 성공 시 JSON 응답 출력
    print("요청 성공 (Status Code: 200)")
    # print(response.json()) # 전체 JSON 응답을 보고 싶을 때
    
    # 예시로 응답의 주요 정보만 출력
    data = response.json()
    if 'routes' in data and data['routes']:
      print(f"경로 수: {len(data['routes'])}개")
      first_route = data['routes'][0]['summary']
      print(f"총 거리: {first_route['distance']} m")
      print(f"총 시간: {first_route['duration']} 초")
    else:
      print("경로 데이터가 응답에 포함되어 있지 않습니다.")
    
  else:
    # 오류 발생 시 상태 코드 및 내용 출력
    print(f"요청 실패 (Status Code: {response.status_code})")
    print(f"오류 메시지: {response.text}")

except requests.exceptions.RequestException as e:
  # 네트워크 등 요청 자체의 오류 처리
  print(f"요청 중 오류 발생: {e}")

요청 성공 (Status Code: 200)
경로 수: 1개
총 거리: 278 m
총 시간: 79 초


In [55]:
import pandas as pd
import numpy as np

src_df = pd.read_csv('20251014_서울시_버스정류장_주소.csv')

In [56]:
src_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41261 entries, 0 to 41260
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ROUTE_ID  41261 non-null  int64  
 1   노선명       41261 non-null  object 
 2   순번        41261 non-null  int64  
 3   NODE_ID   41261 non-null  int64  
 4   ARS_ID    41261 non-null  int64  
 5   정류소명      41261 non-null  object 
 6   X좌표       41261 non-null  float64
 7   Y좌표       41261 non-null  float64
 8   시         41261 non-null  object 
 9   구         41261 non-null  object 
 10  동         41261 non-null  object 
dtypes: float64(2), int64(4), object(5)
memory usage: 3.5+ MB


In [57]:
src_df['거리'] = np.nan

In [58]:
src_df.head()

,ROUTE_ID,노선명,순번,NODE_ID,ARS_ID,정류소명,X좌표,Y좌표,시,구,동,거리
0,100100407,240,1,106000319,7418,중랑공영차고지.신내역,127.1028,37.613545,서울특별시,중랑구,신내동,NaN
1,100100049,273,1,106000319,7418,중랑공영차고지.신내역,127.1028,37.613545,서울특별시,중랑구,신내동,NaN
2,100100186,2012,1,106000319,7418,중랑공영차고지.신내역,127.1028,37.613545,서울특별시,중랑구,신내동,NaN
3,100100190,2015,1,106000319,7418,중랑공영차고지.신내역,127.1028,37.613545,서울특별시,중랑구,신내동,NaN
4,100100599,2311,1,106000319,7418,중랑공영차고지.신내역,127.1028,37.613545,서울특별시,중랑구,신내동,NaN


In [60]:
import pandas as pd
import requests
import time
import json
from io import StringIO
from math import isnan # NaN 값 비교를 위해 임포트

In [96]:
import requests
import numpy as np

def calcDistanceByApi(originX, originY, destX, destY):
    # 2개스페이스로 된 탭으로 보여줍니다.
    # URL 및 헤더 정보
    url = "https://apis-navi.kakaomobility.com/v1/directions"
    headers = {
        "Authorization": "KakaoAK 2a96ad655ffebec848633bc4779b2bf6"
    }

    # 쿼리 파라미터 (Query Parameters)
    # curl 명령어의 '?' 뒤에 오는 부분입니다.
    params = {
        "origin": f"{originX},{originY},angle={calcAngle(originX, originY, destX, destY)}",
        "destination": f"{destX},{destY}",
        "priority": "DISTANCE", # DISTANCE:최단거리, TIME:최단시간, RECOMMEND:추천
        "car_type": "3", # 1:소형,2:중형,3:대형(33인승이상)
        "summary":"true" #요약정보만
    }

    try:
        # GET 요청 보내기
        response = requests.get(url, headers=headers, params=params)

        # 응답 확인
        if response.status_code == 200:
            # 성공 시 JSON 응답 출력
            #print("요청 성공 (Status Code: 200)")
            # print(response.json()) # 전체 JSON 응답을 보고 싶을 때
            
            # 예시로 응답의 주요 정보만 출력
            data = response.json()
            
            # 🌟 수정 부분: 'routes' 키는 확인되었으나, 'summary' 키가 없을 경우 처리
            if 'routes' in data and data['routes']:
                first_route = data['routes'][0]
                
                # 'routes' 리스트의 첫 번째 요소가 예상대로 'summary'를 가지고 있는지 확인
                try:
                    return first_route['summary']['distance']
                
                except KeyError as e:
                    # 1. 'result_code'가 104인지 확인
                    if 'result_code' in first_route and first_route['result_code'] == 104:
                        # result_code 104: "출발지와 도착지가 5 m 이내"
                        # 요청에 따라 5m를 반환합니다.
                        return 5.0 # 거리 단위는 미터(m)

                    # 2. 기타 KeyError 처리 및 디버깅 출력 (이전과 동일)
                    print("-" * 50)
                    print(f"KeyError 발생: {e} 키가 첫 번째 경로 객체에 없습니다.")
                    print("오류 발생 시의 응답 데이터 (data) 전체:")
                    print(data) 
                    print("-" * 50)
                    return np.nan 
                
            else:
                # 'routes' 키가 없거나 리스트가 비어있을 경우 (경로 탐색 실패 등)
                print("경로 데이터가 응답에 포함되어 있지 않거나 비어있습니다.")
                print("응답 데이터 (data) 전체:")
                print(data)
                return np.nan
            
        else:
            # 오류 발생 시 상태 코드 및 내용 출력
            print(f"요청 실패 (Status Code: {response.status_code})")
            print(f"오류 메시지: {response.text}")

    except requests.exceptions.RequestException as e:
        # 네트워크 등 요청 자체의 오류 처리
        print(f"요청 중 오류 발생: {e}=data{data}")

In [66]:
print("aaa=",calcDistanceByApi(originX, originY, destX, destY))

요청 성공 (Status Code: 200)
aaa= 278


In [76]:
DISTANCE_CACHE = {} 

In [83]:
DISTANCE_CACHE

{(np.float64(37.61354475),
  np.float64(127.1027997),
  np.float64(37.614242),
  np.float64(127.099763)): 278,
 (np.float64(37.614242),
  np.float64(127.099763),
  np.float64(37.6138773),
  np.float64(127.0915286)): 944,
 (np.float64(37.6138773),
  np.float64(127.0915286),
  np.float64(37.61389708),
  np.float64(127.0904185)): 154,
 (np.float64(37.61389708),
  np.float64(127.0904185),
  np.float64(37.6165664),
  np.float64(127.0900173)): 394,
 (np.float64(37.6165664),
  np.float64(127.0900173),
  np.float64(37.616207),
  np.float64(127.091775)): 524,
 (np.float64(37.616207),
  np.float64(127.091775),
  np.float64(37.61525321),
  np.float64(127.0938875)): 212,
 (np.float64(37.61525321),
  np.float64(127.0938875),
  np.float64(37.61468),
  np.float64(127.095217)): 133,
 (np.float64(37.61468),
  np.float64(127.095217),
  np.float64(37.61311489),
  np.float64(127.0961797)): 555,
 (np.float64(37.61311489),
  np.float64(127.0961797),
  np.float64(37.61056948),
  np.float64(127.0959907)): 288

In [132]:
import time
from tqdm import tqdm
import numpy as ny

def calculate_route_distances_with_api(df):
  """
  인덱스 매핑 오류를 수정한 함수입니다. 원본 인덱스를 저장하여 정확히 매핑합니다.
  """
  # 결과를 담을 새로운 컬럼 초기화
  df['거리'] = 0.0

  # '노선명'을 기준으로 그룹화
  grouped = df.groupby('노선명')

  # 각 노선별로 순회하며 거리 계산
  for route_name, route_df in tqdm(grouped, desc="노선별 거리 계산 진행", total=len(grouped)):
    # 🌟 수정 포인트 1: 원래의 데이터프레임 인덱스를 저장합니다.
    original_indices = route_df.index
    
    # 순번 오름차순으로 정렬 후 인덱스 재설정 (계산 편의를 위해 0, 1, 2...로 만듦)
    route_df = route_df.sort_values(by='순번').reset_index(drop=True)
    
    route_distances = []
    
    # 거리 계산 루프 (N-1)
    for i in range(len(route_df) - 1):
      lat1 = route_df.loc[i, 'Y좌표']
      lon1 = route_df.loc[i, 'X좌표']
      lat2 = route_df.loc[i + 1, 'Y좌표']
      lon2 = route_df.loc[i + 1, 'X좌표']

      cache_key = (lat1, lon1, lat2, lon2)
      
      if cache_key in globals()['DISTANCE_CACHE']:
        distance = globals()['DISTANCE_CACHE'][cache_key]
      else:
        time.sleep(0.1)
        distance = calcDistanceByApi(lon1, lat1, lon2, lat2)
        
        is_numeric = isinstance(distance, (int, float))
  
        if is_numeric and not np.isnan(distance):
          globals()['DISTANCE_CACHE'][cache_key] = distance
        
      route_distances.append(distance)
      
    # 마지막 순번의 거리는 0.0
    route_distances.append(0.0)

    # 🌟 수정 포인트 2: 저장해둔 원본 인덱스에 계산된 거리를 매핑합니다.
    df.loc[original_indices, '거리'] = route_distances
  
  return df

In [80]:
target_route_df = src_df[src_df['노선명'] == '240'].copy()

print(f"원본 데이터프레임 행 수: {len(src_df)}")
print(f"필터링된 '240' 노선 데이터프레임 행 수: {len(target_route_df)}")
print("\n필터링된 데이터프레임 (일부):")
print(target_route_df.head())

원본 데이터프레임 행 수: 41261
필터링된 '240' 노선 데이터프레임 행 수: 121

필터링된 데이터프레임 (일부):
       ROUTE_ID  노선명  순번    NODE_ID  ARS_ID              정류소명         X좌표  \
0     100100407  240   1  106000319    7418       중랑공영차고지.신내역  127.102800   
1367  100100407  240   2  106000441    7437           새우개마을입구  127.099763   
1651  100100407  240   3  106000189    7284            봉화초등학교  127.091529   
2253  100100407  240   4  106000190    7285   신내6단지아파트.옹기테마공원  127.090418   
2954  100100407  240   5  106000193    7288  신내우체국.5단지두산대림아파트  127.090017   

            Y좌표      시    구    동  거리  
0     37.613545  서울특별시  중랑구  신내동 NaN  
1367  37.614242  서울특별시  중랑구  신내동 NaN  
1651  37.613877  서울특별시  중랑구  신내동 NaN  
2253  37.613897  서울특별시  중랑구  신내동 NaN  
2954  37.616566  서울특별시  중랑구  신내동 NaN  


In [81]:
target_route_df = calculate_route_distances_with_api(target_route_df.copy())

노선별 거리 계산 진행:   0%|                                                                                                                | 0/1 [00:00<?, ?it/s]

노선별 거리 계산 진행: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.79s/it]


In [82]:
target_route_df

,ROUTE_ID,노선명,순번,NODE_ID,ARS_ID,정류소명,X좌표,Y좌표,시,구,동,거리
0,100100407,240,1,106000319,7418,중랑공영차고지.신내역,127.102800,37.613545,서울특별시,중랑구,신내동,278.0
1367,100100407,240,2,106000441,7437,새우개마을입구,127.099763,37.614242,서울특별시,중랑구,신내동,944.0
1651,100100407,240,3,106000189,7284,봉화초등학교,127.091529,37.613877,서울특별시,중랑구,신내동,154.0
2253,100100407,240,4,106000190,7285,신내6단지아파트.옹기테마공원,127.090418,37.613897,서울특별시,중랑구,신내동,394.0
2954,100100407,240,5,106000193,7288,신내우체국.5단지두산대림아파트,127.090017,37.616566,서울특별시,중랑구,신내동,524.0
...,...,...,...,...,...,...,...,...,...,...,...,...
40448,100100407,240,117,106000188,7283,봉화초등학교,127.091724,37.613761,서울특별시,중랑구,신내동,535.0
40491,100100407,240,118,106000186,7281,신내7단지아파트.홈플러스,127.093887,37.615253,서울특별시,중랑구,신내동,133.0
40532,100100407,240,119,106000257,7354,중랑소방서,127.095217,37.614680,서울특별시,중랑구,신내동,732.0
40574,100100407,240,120,106000241,7337,중랑공영차고지,127.102023,37.613645,서울특별시,중랑구,신내동,148.0


In [133]:
src_df_with_distance = calculate_route_distances_with_api(src_df.copy())

노선별 거리 계산 진행:  10%|██████████▍                                                                                         | 74/705 [00:00<00:00, 735.72it/s]

--------------------------------------------------
KeyError 발생: 'summary' 키가 첫 번째 경로 객체에 없습니다.
오류 발생 시의 응답 데이터 (data) 전체:
{'trans_id': '0199ea5977a47ed3abe619d45ad8f36c', 'routes': [{'result_code': 106, 'result_msg': '도착 지점 주변의 도로에 유고 정보(교통 장애)가 있음'}]}
--------------------------------------------------
--------------------------------------------------
KeyError 발생: 'summary' 키가 첫 번째 경로 객체에 없습니다.
오류 발생 시의 응답 데이터 (data) 전체:
{'trans_id': '0199ea59788573b2afdc82c9acfdc90b', 'routes': [{'result_code': 105, 'result_msg': '시작 지점 주변의 도로에 유고 정보(교통 장애)가 있음'}]}
--------------------------------------------------
--------------------------------------------------
KeyError 발생: 'summary' 키가 첫 번째 경로 객체에 없습니다.
오류 발생 시의 응답 데이터 (data) 전체:
{'trans_id': '0199ea5979a27bdf950ee5d6965731b2', 'routes': [{'result_code': 106, 'result_msg': '도착 지점 주변의 도로에 유고 정보(교통 장애)가 있음'}]}
--------------------------------------------------


노선별 거리 계산 진행:  21%|████████████████████▊                                                                              | 148/705 [00:01<00:04, 117.68it/s]

--------------------------------------------------
KeyError 발생: 'summary' 키가 첫 번째 경로 객체에 없습니다.
오류 발생 시의 응답 데이터 (data) 전체:
{'trans_id': '0199ea597a867847a96802b92dd30289', 'routes': [{'result_code': 105, 'result_msg': '시작 지점 주변의 도로에 유고 정보(교통 장애)가 있음'}]}
--------------------------------------------------
--------------------------------------------------
KeyError 발생: 'summary' 키가 첫 번째 경로 객체에 없습니다.
오류 발생 시의 응답 데이터 (data) 전체:
{'trans_id': '0199ea597b797070b9d8c2a53df722f7', 'routes': [{'result_code': 106, 'result_msg': '도착 지점 주변의 도로에 유고 정보(교통 장애)가 있음'}]}
--------------------------------------------------
--------------------------------------------------
KeyError 발생: 'summary' 키가 첫 번째 경로 객체에 없습니다.
오류 발생 시의 응답 데이터 (data) 전체:
{'trans_id': '0199ea597c4b772eb55b4500beda9b50', 'routes': [{'result_code': 105, 'result_msg': '시작 지점 주변의 도로에 유고 정보(교통 장애)가 있음'}]}
--------------------------------------------------
--------------------------------------------------
KeyError 발생: 'summary' 키가 첫 번째 경로 객체에

노선별 거리 계산 진행:  26%|█████████████████████████▉                                                                          | 183/705 [00:03<00:12, 41.04it/s]

...

--------------------------------------------------
KeyError 발생: 'summary' 키가 첫 번째 경로 객체에 없습니다.
오류 발생 시의 응답 데이터 (data) 전체:
{'trans_id': '0199ea599d307c51b13286b519a6f8b4', 'routes': [{'result_code': 106, 'result_msg': '도착 지점 주변의 도로에 유고 정보(교통 장애)가 있음'}]}
--------------------------------------------------
--------------------------------------------------
KeyError 발생: 'summary' 키가 첫 번째 경로 객체에 없습니다.
오류 발생 시의 응답 데이터 (data) 전체:
{'trans_id': '0199ea599e75702187ea257fb1d36717', 'routes': [{'result_code': 106, 'result_msg': '도착 지점 주변의 도로에 유고 정보(교통 장애)가 있음'}]}
--------------------------------------------------


노선별 거리 계산 진행:  87%|█████████████████████████████████████████████████████████████████████████████████████▋             | 610/705 [00:10<00:00, 149.37it/s]

--------------------------------------------------
KeyError 발생: 'summary' 키가 첫 번째 경로 객체에 없습니다.
오류 발생 시의 응답 데이터 (data) 전체:
{'trans_id': '0199ea599f5673668d9ca635139268d6', 'routes': [{'result_code': 105, 'result_msg': '시작 지점 주변의 도로에 유고 정보(교통 장애)가 있음'}]}
--------------------------------------------------
--------------------------------------------------
KeyError 발생: 'summary' 키가 첫 번째 경로 객체에 없습니다.
오류 발생 시의 응답 데이터 (data) 전체:
{'trans_id': '0199ea59a03b7b25b5e401ce415e92bc', 'routes': [{'result_code': 106, 'result_msg': '도착 지점 주변의 도로에 유고 정보(교통 장애)가 있음'}]}
--------------------------------------------------
--------------------------------------------------
KeyError 발생: 'summary' 키가 첫 번째 경로 객체에 없습니다.
오류 발생 시의 응답 데이터 (data) 전체:
{'trans_id': '0199ea59a1077c6485dcfc8de98ff192', 'routes': [{'result_code': 105, 'result_msg': '시작 지점 주변의 도로에 유고 정보(교통 장애)가 있음'}]}
--------------------------------------------------
--------------------------------------------------
KeyError 발생: 'summary' 키가 첫 번째 경로 객체에

노선별 거리 계산 진행: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 705/705 [00:11<00:00, 61.57it/s]

--------------------------------------------------
KeyError 발생: 'summary' 키가 첫 번째 경로 객체에 없습니다.
오류 발생 시의 응답 데이터 (data) 전체:
{'trans_id': '0199ea59a2aa749e8dc73695a58d1fef', 'routes': [{'result_code': 105, 'result_msg': '시작 지점 주변의 도로에 유고 정보(교통 장애)가 있음'}]}
--------------------------------------------------


In [134]:
src_df_with_distance

,ROUTE_ID,노선명,순번,NODE_ID,ARS_ID,정류소명,X좌표,Y좌표,시,구,동,거리
0,100100407,240,1,106000319,7418,중랑공영차고지.신내역,127.102800,37.613545,서울특별시,중랑구,신내동,278.0
1,100100049,273,1,106000319,7418,중랑공영차고지.신내역,127.102800,37.613545,서울특별시,중랑구,신내동,812.0
2,100100186,2012,1,106000319,7418,중랑공영차고지.신내역,127.102800,37.613545,서울특별시,중랑구,신내동,317.0
3,100100190,2015,1,106000319,7418,중랑공영차고지.신내역,127.102800,37.613545,서울특별시,중랑구,신내동,812.0
4,100100599,2311,1,106000319,7418,중랑공영차고지.신내역,127.102800,37.613545,서울특별시,중랑구,신내동,317.0
...,...,...,...,...,...,...,...,...,...,...,...,...
41256,100100589,N61,183,116000142,17233,개봉역,126.856720,37.493978,서울특별시,구로구,개봉동,1318
41257,100100589,N61,184,116000144,17235,개봉1동사거리.개봉푸르지오아파트,126.846791,37.501413,서울특별시,구로구,개봉동,714
41258,100100589,N61,185,114000125,15228,신정학마을아파트,126.843963,37.507421,서울특별시,양천구,신정동,304
41259,100100589,N61,186,114000257,15362,신정현대아파트.서부트럭터미널,126.842007,37.509011,서울특별시,양천구,신정동,574


In [110]:

src_df_with_distance[src_df_with_distance.isna().any(axis=1)]

,ROUTE_ID,노선명,순번,NODE_ID,ARS_ID,정류소명,X좌표,Y좌표,시,구,동,거리
45,116900006,구로07,1,116900277,17601,오류동역2번출구,126.843790,37.493629,서울특별시,구로구,오류동,None
46,100100437,771,1,219001178,36350,대화동종점,126.759705,37.686368,경기도,고양시 일산서구,대화동,None
47,100100359,7728,1,219001178,36350,대화동종점,126.759705,37.686368,경기도,고양시 일산서구,대화동,None
48,100100440,700,1,219001178,36350,대화동종점,126.759705,37.686368,경기도,고양시 일산서구,대화동,None
49,100100511,760,1,219001178,36350,대화동종점,126.759705,37.686368,경기도,고양시 일산서구,대화동,None
...,...,...,...,...,...,...,...,...,...,...,...,...
41197,111000017,N72,161,113000014,14104,월드컵경기장남측.월드컵공원,126.896317,37.566664,서울특별시,마포구,성산동,None
41230,111000016,N75,167,111000096,12184,연신내역.로데오거리,126.919996,37.617821,서울특별시,은평구,대조동,None
41235,111000017,N72,168,113000021,14111,DMC첨단산업센터,126.885192,37.585347,서울특별시,마포구,상암동,None
41236,111000017,N72,169,111000224,12314,덕은교.은평차고지앞,126.885659,37.588594,서울특별시,은평구,수색동,None


In [138]:
src_df_with_distance.to_csv('20251015_서울시_버스정류장_주소_거리.csv', index=False, encoding='utf-8')


In [136]:
print("DISTANCE_CACHE=",len(DISTANCE_CACHE))

DISTANCE_CACHE= 17229


In [123]:
import pickle

def save_cache_to_pickle(cache, filename="distance_cache.pkl"):
  with open(filename, 'wb') as f: # 'wb'는 Write Binary 모드
    pickle.dump(cache, f)
  print(f"캐시가 '{filename}'에 Pickle 형식으로 저장되었습니다.")

In [137]:
save_cache_to_pickle(DISTANCE_CACHE, "20251015_DISTANCE_CACHE.pkl")

캐시가 '20251015_DISTANCE_CACHE.pkl'에 Pickle 형식으로 저장되었습니다.


In [91]:
import pickle

def load_cache_from_pickle(filename="20251015_DISTANCE_CACHE.pkl"):
  try:
    with open(filename, 'rb') as f: # 'rb'는 Read Binary 모드
      loaded_cache = pickle.load(f)
    print(f"캐시를 '{filename}'에서 성공적으로 불러왔습니다.")
    return loaded_cache
  except FileNotFoundError:
    print(f"경고: '{filename}' 파일을 찾을 수 없습니다. 빈 캐시를 반환합니다.")
    return {}

In [125]:
# 1. 프로그램 시작 시 캐시 로드
DISTANCE_CACHE_LOADED = load_cache_from_pickle("20251015_DISTANCE_CACHE.pkl") 


캐시를 '20251015_DISTANCE_CACHE.pkl'에서 성공적으로 불러왔습니다.


In [126]:
print("DISTANCE_CACHE=",len(DISTANCE_CACHE_LOADED))

DISTANCE_CACHE= 17239


In [112]:
DISTANCE_CACHE_LOADED

{(np.float64(37.61354475),
  np.float64(127.1027997),
  np.float64(37.614242),
  np.float64(127.099763)): 278,
 (np.float64(37.614242),
  np.float64(127.099763),
  np.float64(37.6138773),
  np.float64(127.0915286)): 944,
 (np.float64(37.6138773),
  np.float64(127.0915286),
  np.float64(37.61389708),
  np.float64(127.0904185)): 154,
 (np.float64(37.61389708),
  np.float64(127.0904185),
  np.float64(37.6165664),
  np.float64(127.0900173)): 394,
 (np.float64(37.6165664),
  np.float64(127.0900173),
  np.float64(37.616207),
  np.float64(127.091775)): 524,
 (np.float64(37.616207),
  np.float64(127.091775),
  np.float64(37.61525321),
  np.float64(127.0938875)): 212,
 (np.float64(37.61525321),
  np.float64(127.0938875),
  np.float64(37.61468),
  np.float64(127.095217)): 133,
 (np.float64(37.61468),
  np.float64(127.095217),
  np.float64(37.61311489),
  np.float64(127.0961797)): 555,
 (np.float64(37.61311489),
  np.float64(127.0961797),
  np.float64(37.61056948),
  np.float64(127.0959907)): 288

In [127]:
import math


def check_nan_in_dict(d):
    for value in d.values():
        # 값이 float 타입이고, math.isnan()을 통과하는지 확인
        if isinstance(value, float) and math.isnan(value):
            print()
            return True
    return False

result = check_nan_in_dict(DISTANCE_CACHE)

print(f"DISTANCE_CACHE에 NaN이 있습니까?: {result}")


DISTANCE_CACHE에 NaN이 있습니까?: True


In [128]:
cleaned_cache = {}

for key, value in DISTANCE_CACHE.items():
    # numpy.isnan()을 사용하기 위해, 값이 숫자 타입인지 먼저 확인합니다.
    is_numeric = isinstance(value, (int, float))
    
    # 숫자인 경우: np.isnan()으로 NaN이 아닌지 확인
    # 숫자가 아닌 경우: NaN이 아니라고 간주 (문자열 등)
    if is_numeric and np.isnan(value):
        continue  # NaN인 경우 건너뛰고 (삭제)
    else:
        cleaned_cache[key] = value # NaN이 아니면 유지

# 만약 NumPy를 사용하지 않는다면 math 모듈을 사용하여 구현할 수도 있습니다.

#print(f"NaN 삭제 후 딕셔너리: {cleaned_cache}")

result = check_nan_in_dict(cleaned_cache)

print(f"DISTANCE_CACHE에 NaN이 있습니까?: {result}")

DISTANCE_CACHE에 NaN이 있습니까?: False


In [129]:
print(len(cleaned_cache))

17229


In [130]:
DISTANCE_CACHE = cleaned_cache

In [131]:
print(len(DISTANCE_CACHE))

17229
